In [ ]:
#GitHub Link:https://github.com/arpit-shrivastava-2020/Deep-Learning-5th-Lab-Assignment.git

In [1]:
import numpy as np
import random

def get_initial_loss(vocab_size, seq_length):
    return -np.log(1.0/vocab_size)*seq_length

def smooth(loss, cur_loss):
    return loss * 0.999 + cur_loss * 0.001

def print_sample(sample_ix, ix_to_char):
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    txt = txt[0].upper() + txt[1:]  # capitalize first character
    print('%s' % (txt, ), end='')


def initialize_parameters(n_a, n_x, n_y):
    Wax = np.random.randn(n_a, n_x)*0.01  # input to hidden
    Waa = np.random.randn(n_a, n_a)*0.01  # hidden to hidden
    Wya = np.random.randn(n_y, n_a)*0.01  # hidden to output
    ba = np.zeros((n_a, 1))  # hidden bias
    by = np.zeros((n_y, 1))  # output bias

    parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "ba": ba, "by": by}

    return parameters


def update_parameters(parameters, gradients, lr):

    parameters['Wax'] += -lr * gradients['dWax']
    parameters['Waa'] += -lr * gradients['dWaa']
    parameters['Wya'] += -lr * gradients['dWya']
    parameters['ba'] += -lr * gradients['db']
    parameters['by'] += -lr * gradients['dby']
    
    return parameters

def rnn_cell_forward(xt, a_prev, parameters):
    
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]

    a_next = np.tanh(np.dot(Wax, xt) + np.dot(Waa, a_prev) + ba)
    yt_pred = softmax(np.dot(Wya, a_next) + by)

    cache = (a_next, a_prev, xt, parameters)

    return a_next, yt_pred, cache

def rnn_forward(X, Y, a0, parameters, vocab_size=27):
    x, a, y_hat = {}, {}, {}
    a[-1] = np.copy(a0)
    loss = 0
    for t in range(len(X)):
        x[t] = np.zeros((vocab_size, 1))
        if (X[t] != None):
            x[t][X[t]] = 1
        a[t], y_hat[t], _ = rnn_cell_forward(x[t], a[t-1],parameters)
        loss -= np.log(y_hat[t][Y[t], 0])
    cache = (y_hat, a, x)
    return loss, cache

def rnn_cell_backward(dy, gradients, parameters, x, a, a_prev):
    gradients['dWya'] += np.dot(dy, a.T)
    gradients['dby'] += dy
    da = np.dot(parameters['Wya'].T, dy) +         gradients['da_next']  # backprop into h
    daraw = (1 - a * a) * da  # backprop through tanh nonlinearity
    gradients['db'] += daraw
    gradients['dWax'] += np.dot(daraw, x.T)
    gradients['dWaa'] += np.dot(daraw, a_prev.T)
    gradients['da_next'] = np.dot(parameters['Waa'].T, daraw)
    return gradients


def rnn_backward(X, Y, parameters, cache):
    gradients = {}
    (y_hat, a, x) = cache
    Waa, Wax, Wya, by, ba = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['ba']
    gradients['dWax'], gradients['dWaa'], gradients['dWya'] = np.zeros_like(Wax), np.zeros_like(Waa), np.zeros_like(Wya)
    gradients['db'], gradients['dby'] = np.zeros_like(ba), np.zeros_like(by)
    gradients['da_next'] = np.zeros_like(a[0])

    for t in reversed(range(len(X))):
        dy = np.copy(y_hat[t])
        dy[Y[t]] -= 1
        gradients = rnn_cell_backward(
            dy, gradients, parameters, x[t], a[t], a[t-1])
    
    return gradients, a


def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)


def clip(gradients, maxValue):
    dWaa, dWax, dWya, db, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['db'], gradients['dby']
   
    for gradient in [dWax, dWaa, dWya, db, dby]:
        np.clip(gradient, -maxValue, maxValue, out=gradient)
    
    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "db": db, "dby": dby}
    
    return gradients


def sample(parameters, char_to_ix, seed):
    Waa, Wax, Wya, by, ba = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['ba']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]    
    x = np.zeros((vocab_size, 1))
    a_prev = np.zeros((n_a, 1))
    indices = []
    
    idx = -1 
    
    counter = 0
    newline_character = char_to_ix['\n']
    
    while (idx != newline_character and counter != 50):
        a = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + ba)
        z = np.dot(Wya, a) + by
        y = softmax(z)
            
        idx = np.random.choice(list(range(vocab_size)), p=y.ravel())

        indices.append(idx)
        
        x = np.zeros((vocab_size, 1))
        x[idx] = 1
        
        a_prev = a
        
        counter +=1
        
    if (counter == 50):
        indices.append(char_to_ix['\n'])
    
    return indices


def optimize(X, Y, a_prev, parameters, learning_rate = 0.01):
    loss, cache = rnn_forward(X, Y, a_prev, parameters)
    
    gradients, a = rnn_backward(X, Y, parameters, cache)
    
    gradients = clip(gradients, 5)
    
    parameters = update_parameters(parameters, gradients, learning_rate)
        
    return loss, gradients, a[len(X)-1]


def model(data, ix_to_char, char_to_ix, num_iterations = 35000, n_a = 50, dino_names = 7, vocab_size = 27):
    
    n_x, n_y = vocab_size, vocab_size
    
    parameters = initialize_parameters(n_a, n_x, n_y)
    
    loss = get_initial_loss(vocab_size, dino_names)
    
    with open("dinos.txt") as f:
        examples = f.readlines()
    examples = [x.lower().strip() for x in examples]
    
    np.random.seed(0)
    np.random.shuffle(examples)
    
    a_prev = np.zeros((n_a, 1))
    
    for j in range(num_iterations):
        
        index = j % len(examples)
        X = [None] + [char_to_ix[ch] for ch in examples[index]] 
        Y = X[1:] + [char_to_ix["\n"]]
        
        curr_loss, gradients, a_prev = optimize(X, Y, a_prev, parameters)
        
        loss = smooth(loss, curr_loss)

        if j % 2000 == 0:
            
            print('Iteration: %d, Loss: %f' % (j, loss) + '\n')
            
            seed = 0
            for name in range(dino_names):
                
                sampled_indices = sample(parameters, char_to_ix, seed)
                txt = ''.join(ix_to_char[ix] for ix in sampled_indices)
                txt = txt[0].upper() + txt[1:]  # capitalize first character
                print('%s' % (txt, ), end='')
                
                seed += 1  
      
            print('\n')
        
    return parameters

data = open('dinos.txt', 'r').read()
data= data.lower()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))

char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }

model(data, ix_to_char, char_to_ix)

There are 19909 total characters and 27 unique characters in your data.
Iteration: 0, Loss: 23.087336

Co
Eyhqgyshubjerbjgmis
Qfguyhvbqtdkxb
Okuxhjmozrjgycsxqkeisqywvxgijorg
Jhywwqinubuavuapmdctozzzcyflteoebci
Kqjbagdzduoomziuojypckurxgtloyzwiwqk
Tmutuhunlr


Iteration: 2000, Loss: 27.882500

Amychangwsacrusycripshurashudncqrusayraalrptacsaus
Gonqneleprus
Anisaurus
Anqoshorostisaurur
Ranelocsaurus
Ehlgwyrovtonackytabtonos
Calceradenderzotibheosaurus


Iteration: 4000, Loss: 25.878239

Gomatosaurus
Minvhytosaurus
Angthosaurus
Vourosaurus
Akaceosaurus
Ynainnphosaurus
Rugoruous


Iteration: 6000, Loss: 24.652644

Lorspgnomeortoraszurateostos
Lildinhosaune
Sisnrovrosavosaurus
Stetintynlerrorosaurus
Deorostosauri
Enoeosaurusqsauhekerossaurosaurus
Ahusaurus


Iteration: 8000, Loss: 24.146473

Annspalnatoruitelodon
Ustanopemus
Asalnplosceromeichamnan
Lrochrator
Alephon
Hutesaurus
Rropinisaurus


Iteration: 10000, Loss: 23.899045

Skyaramasaurosaurus
Ngycama
Ntorus
Opaovonatous
Angolosaurus
Yh

{'Waa': array([[-0.13480882,  0.34505466,  0.18556581, ...,  0.47646669,
         -0.12709853,  0.19799195],
        [-0.32431566, -0.52469842, -0.02991147, ...,  0.28943028,
          0.18537565,  0.44393654],
        [ 0.53009569,  0.01772332, -0.20278475, ...,  0.0162098 ,
          0.14523874,  0.15921288],
        ...,
        [-0.03441569, -0.11002757,  0.44711859, ..., -0.01999878,
         -0.04662057,  0.63608135],
        [-0.03249371, -0.06668051,  0.14008852, ...,  0.18737067,
         -0.22422363, -0.15068433],
        [ 0.07076852,  0.40185271, -0.22940305, ..., -0.39883206,
         -0.00433171,  0.44679378]]),
 'Wax': array([[ 2.93592659e-03,  6.12294251e-01,  1.54743617e-01, ...,
         -2.44025855e-02, -6.61036913e-01, -6.17074464e-01],
        [ 4.79402704e-03,  6.27523789e-01,  7.12554976e-01, ...,
          5.34954730e-01,  1.12468936e+00, -8.06353444e-01],
        [ 3.26520651e-03,  1.97227898e-01,  5.36714606e-01, ...,
         -5.24763469e-01, -5.82123534e-02,